In [1]:
import h5py    
import numpy as np    
import pandas as pd
from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import z_at_value
import astropy.units as u
from astropy import constants as const

In [2]:
yrsec = (1*u.yr).to(u.s).value # 1 year in secs
solar_mass = const.M_sun.value #in Kgs

#hubble_time = (1/cosmo.H(0)).to(u.yr).value #Hubble time = 1/H0 
age_of_the_universe = cosmo.age(0).to(u.yr).value

In [27]:
#parent folder
pfolder = "/Users/pranavsatheesh/Triples/Github/"
import sys
sys.path.append(pfolder)

file_name = pfolder + "Illustris_Data/mbhb-evolution_no-ecc_lc-full-0.6.hdf5"
f1 = h5py.File(file_name,'r') 
Ms = np.array(f1['evolution']['masses'])
t = np.array(f1['evolution']['times'])
r = np.array(f1['evolution']['sep'])

In [28]:
Nbinary = len(Ms)
M1 = Ms[:,0]
M2 = Ms[:,1]

t_list = []
for i in range(Nbinary):
    t_list.append(t[i][np.argwhere(r[i]==0)[0][0]]) 

M1 = M1/(solar_mass*10**3) #mass in gs (verify this)
M2 = M2/(solar_mass*10**3)#mass in gs (verify this)
t_list = t_list/yrsec #convert to years

merger = [] #to indicate if the binary actually merges before hubble time
z_binary = []

for time in t_list:
    if time >= age_of_the_universe:
        #these black holes aren't merging
        merger.append("No")
        z_binary.append(0)
    else:
        merger.append("Yes")
        z_binary.append(z_at_value(cosmo.age,(time/10**9)*u.Gyr,zmin=1e-9).value)

df = pd.DataFrame([M1,M2,t_list,merger,z_binary])
df = df.transpose()
df.columns = ['M1', 'M2', 't_merger','Merger','Redshift']



In [29]:
#binary ids
mergers = np.load(pfolder+'Illustris_Data/ill-1_blackhole_mergers_fixed.npz')
indexes = f1['evolution']['val_inds'][:]
binary_ids = mergers['ids'][indexes]

Triple_df = pd.read_csv("../Data/triple_data_ill.csv") #the triples data file from find_triples

In [30]:
bh1id1 = Triple_df["BH1_ID1"].to_numpy()
bh1id2 = Triple_df["BH1_ID2"].to_numpy()
bh2id1 = Triple_df["BH2_ID1"].to_numpy()
bh2id2 = Triple_df["BH2_ID2"].to_numpy()

In [31]:
def searchID(id):

    #to search if the binary ids belong to any triples, indicating a triple interaction

    tf1 = id in bh1id1
    tf2 = id in bh1id2
    tf3 = id in bh2id1
    tf4 = id in bh2id2

    tf = tf1 + tf2 + tf3 + tf4

    
    return tf 

#type = [] #"iso" for binaries that undergo isolated evolution. "trip" for the ones that have some kind of triple interaction"


In [37]:
bh_type = [] #"iso" for binaries that undergo isolated evolution. "trip" for the ones that have some kind of triple interaction"

for i in range(Nbinary):

    search1 = searchID(binary_ids[i][0])
    search2 = searchID(binary_ids[i][1])

    search = search1 + search2
    if(search == 0):
        bh_type.append("iso")
    else:
        print(search)
        bh_type.append("trip")
        print("trip-found")

df["Type"] = bh_type

1
trip-found
1
trip-found
1
trip-found
3
trip-found
2
trip-found
2
trip-found
2
trip-found
3
trip-found
3
trip-found
3
trip-found
3
trip-found
2
trip-found
2
trip-found
2
trip-found
1
trip-found
2
trip-found
1
trip-found
1
trip-found
2
trip-found
2
trip-found
1
trip-found
3
trip-found
2
trip-found
3
trip-found
3
trip-found
1
trip-found
1
trip-found
3
trip-found
1
trip-found
1
trip-found
2
trip-found
2
trip-found
1
trip-found
1
trip-found
1
trip-found
1
trip-found
2
trip-found
2
trip-found
1
trip-found
1
trip-found
1
trip-found
3
trip-found
2
trip-found
2
trip-found
3
trip-found
2
trip-found
1
trip-found
2
trip-found
1
trip-found
3
trip-found
2
trip-found
2
trip-found
2
trip-found
3
trip-found
1
trip-found
2
trip-found
3
trip-found
1
trip-found
1
trip-found
1
trip-found
1
trip-found
3
trip-found
1
trip-found
1
trip-found
1
trip-found
3
trip-found
1
trip-found
3
trip-found
2
trip-found
3
trip-found
2
trip-found
3
trip-found
1
trip-found
3
trip-found
2
trip-found
1
trip-found
3
trip-found

In [33]:
df[df["Type"] == "trip"]

,M1,M2,t_merger,Merger,Redshift,Type
57,1907656.25,1383896.306818,1686323031.711682,Yes,3.75142,trip
68,32875568.181818,12541534.090909,2055998677.743484,Yes,3.158812,trip
73,52783948.863636,8207173.295455,1988184838.474371,Yes,3.253781,trip
86,10597187.5,2885980.113636,325572663676.466003,No,0,trip
88,1159612.215909,1078595.170455,10418882111.340479,Yes,0.288184,trip
...,...,...,...,...,...,...
9120,123969886.363636,17152130.681818,56329989493.436363,No,0,trip
9145,142481534.090909,1854744.318182,14577371512.906393,No,0,trip
9156,12865482.954545,10526818.181818,13616171751.93148,Yes,0.010912,trip
9168,309325284.090909,2218934.659091,14772363476.009422,No,0,trip


In [34]:
type(bh1id1[0]) #IDs of BHs in triples

numpy.float64

In [35]:
type(binary_ids[0,0]) #IDs of BHs in binaries

numpy.uint64

In [36]:
for bin_id in binary_ids[:,0]:

    if(bin_id==bh1id1[0]):
        print("BH in triple ID:", bh1id1[0])
        print("BH in binary ID:", bin_id)

        print("BH in triple ID after conversion to int:",int(bh1id1[0]))

BH in triple ID: 9.22337203709081e+18
BH in binary ID: 9223372037090809465
BH in triple ID after conversion to int: 9223372037090809856
